In [6]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import base64
from openai import OpenAI


[MYCODE] .env 파일 로드 
- opeanAIKey Load
- 서버에 환경셋팅

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

[MYCODE] 모델 설정 

In [9]:
llm = OpenAI()

In [182]:
me_url = "https://static.talented.co.kr/user_data/mjin5563/popol_detail/6fc201f4-59ea-4aef-b32e-819be7d015d8.jpeg"
item_url = "https://image-cdn.trenbe.com/product-images/1732811889714_70d413f684b3a0ccd0f38f25d83072f6_0.jpeg"
sub_item_url = "https://thumbnail10.coupangcdn.com/thumbnails/remote/492x492ex/image/vendor_inventory/ebd7/2ee15ae9f8067bcef856defc02b8e9814ca6b37af7d47e593e4306515509.jpg"

prompt = """
당신은 유명한 패션 디자이너 AI입니다.

여러 개의 이미지가 있으며, 각 이미지에는 설명(description)이 부여되어 있습니다. 각 설명을 참고하여 다음 질문에 답변하세요.

**질문 1:**
**me**의 키는 몇인가요?
- **답변:** 180

**질문 2:**
**me**의 머리 스타일은 무엇인가요?
- **답변:** 짧다.

**질문 3:**
**me**의 손은 무엇이 있나요?
- **답변:** 팔찌

**질문 4:**
**me**는 상체에 어떤 옷을 입었나요?
- **답변:** 검정색 티셔츠

**질문 5:**
**me**는 하체는 어떤 옷을 입었나요?
- **답변:** 검정색 청바지

**질문 6:**
**me**의 성별은 무엇인가요?
- **답변:** 남자

**중간 요약:**
키가 180 정도 되는 남자는 머리가 짧고, 손에는 팔찌를 착용했으며, 상체는 검정색 티셔츠, 하체는 검정색 청바지를 입고 있습니다.

**me** 이미지에 어울리는 패션 아이템 세가지를 **#구매한 아이템** 참고해서 추천 해주세요
#구매한 아이템
**구매한 첫번째 아이템**
**me**가 구매한 **sell_item1**은 어떤 물건이며, 어떤 색과 모양을 가지고 있나요?
- **답변:** 물건: 모자, 색상: 빨간색, 모양: 체크무늬

**구매한 두번째 아이템**
**me**가 구매한 **sell_item2**은 어떤 물건이며, 어떤 색과 모양을 가지고 있나요?
- **답변:** 물건: 점퍼, 색상: 파란색, 모양: 민무늬

# 최종답변 포맷
{
  "recommended_items": [
    {
      "item": "모자",
      "color": "빨간색",
      "style": "심플한 디자인"
    },
    {
      "item": "가방",
      "color": "갈색",
      "style": "캔버스 재질의 쇼퍼백"
    },
    {
      "item": "패션 바지",
      "color": "빨간색",
      "style": "편안한 와이드 팬츠"
    }
  ]
  "recommended_items_description" : "검정색 상의와 빨간색바지가 조화롭고 가방은 좀더 개성을 보여줄 것 같네요. 머리 길이가 짧아 모자도 잘 어울리는 패션이될 것 같네요"
}

//-----------------------

**me** 이미지에 어울리는 패션 아이템 세가지를 **#구매한 아이템** 참고해서 추천해주세요. 

**#구매한 아이템
**구매한 첫번째 아이템**
**me**가 구매한 **sell_item1**은 어떤 물건이며, 어떤 색과 모양을 가지고 있나요?
- **답변:** 

**구매한 두번째 아이템**
**me**가 구매한 **sell_item2**은 어떤 물건이며, 어떤 색과 모양을 가지고 있나요?
- **답변:** 

- **최종답변:** 

"""

response = llm.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                
                {"type": "text", "text": prompt},                    
                {"type": "image_url", "image_url" :{"url": me_url, "description" : "me"}}, 
                {"type": "image_url", "image_url" :{"url": item_url, "description" : "sell_item1"}}, 
                {"type": "image_url", "image_url" :{"url": sub_item_url, "description" : "sell_item2"}}, 
            ],
        }
    ],
    max_tokens=300,
)

In [88]:
import json

In [184]:
content = response.choices[0].message.content
json.loads(content)

{'recommended_items': [{'item': '모자', 'color': '빨간색', 'style': '심플한 디자인'},
  {'item': '가방', 'color': '갈색', 'style': '캔버스 재질의 쇼퍼백'},
  {'item': '패션 바지', 'color': '빨간색', 'style': '편안한 와이드 팬츠'}],
 'recommended_items_description': '흰색 상의와 잘 어울리는 빨간색 팬츠는 스타일리시하면서도 편안함을 더해 줄 것입니다. 그리고 갈색의 쇼퍼백을 매치하여 개성을 더하면 좋겠네요. 마지막으로, 간단한 빨간색 모자도 스타일에 포인트를 줄 수 있어 추천합니다.'}